In [88]:
# SQL Cheatsheet Notebook
# This notebook serves as a practical SQL reference and practice guide.
# It includes:
# - Creation of 4 toy tables for hands-on experimentation
# - A 10-question quiz to test SQL skills

In [19]:
import sqlite3
import pandas as pd

In [29]:
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

### Create toy tables and insert synthetic values

In [ ]:
customer_table_query = \
"""
CREATE TABLE customers (
    cust_id INTEGER PRIMARY KEY,
    name TEXT,
    country TEXT,
    signup_date TEXT,
    vip BOOLEAN
)
"""

product_table_query = \
"""
CREATE TABLE products (
    prod_id INTEGER PRIMARY KEY,
    name TEXT,
    category TEXT,
    price REAL
)
"""

order_table_query = \
"""
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    cust_id INTEGER,
    order_date TEXT,
    status TEXT,
    FOREIGN KEY (cust_id) REFERENCES customers(cust_id)
)
"""

order_records_table_query = \
"""
CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    prod_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (prod_id) REFERENCES products(prod_id)
)
"""

cursor.execute(customer_table_query)
cursor.execute(product_table_query)
cursor.execute(order_table_query)
cursor.execute(order_records_table_query)

cursor.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?)", [
    (1, "Alice", "USA", "2022-01-10", 1),
    (2, "Bob", "UK", "2022-02-15", 0),
    (3, "Charlie", "USA", "2022-03-01", 0),
    (4, "Diana", "Germany", "2022-04-12", 1),
    (5, "Eve", "Spain", "2022-05-05", 0),
])

cursor.executemany("INSERT INTO products VALUES (?, ?, ?, ?)", [
    (1, "Laptop", "Electronics", 1200),
    (2, "Headphones", "Electronics", 150),
    (3, "Coffee Machine", "Home", 300),
    (4, "Desk", "Furniture", 200),
    (5, "Chair", "Furniture", 100),
])

cursor.executemany("INSERT INTO orders VALUES (?, ?, ?, ?)", [
    (101, 1, "2023-01-10", "Completed"),
    (102, 2, "2023-01-12", "Completed"),
    (103, 1, "2023-02-01", "Pending"),
    (104, 3, "2023-02-05", "Completed"),
    (105, 4, "2023-02-20", "Completed"),
])

cursor.executemany("INSERT INTO order_items VALUES (?, ?, ?, ?)", [
    (1, 101, 1, 1),   
    (2, 101, 2, 2),   
    (3, 102, 3, 1),   
    (4, 103, 4, 1),  
    (5, 104, 5, 4),   
    (6, 105, 1, 2),   
])

connection.commit()

In [31]:
def run_query(query):
    return pd.read_sql_query(query, connection)

In [42]:
run_query("select * from customers;")

,cust_id,name,country,signup_date,vip
0,1,Alice,USA,2022-01-10,1
1,2,Bob,UK,2022-02-15,0
2,3,Charlie,USA,2022-03-01,0
3,4,Diana,Germany,2022-04-12,1
4,5,Eve,Spain,2022-05-05,0


In [34]:
run_query("select * from orders")

,order_id,cust_id,order_date,status
0,101,1,2023-01-10,Completed
1,102,2,2023-01-12,Completed
2,103,1,2023-02-01,Pending
3,104,3,2023-02-05,Completed
4,105,4,2023-02-20,Completed


In [40]:
run_query("select * from products;")


,prod_id,name,category,price
0,1,Laptop,Electronics,1200.0
1,2,Headphones,Electronics,150.0
2,3,Coffee Machine,Home,300.0
3,4,Desk,Furniture,200.0
4,5,Chair,Furniture,100.0


In [39]:
run_query("select * from order_items;")


,order_item_id,order_id,prod_id,quantity
0,1,101,1,1
1,2,101,2,2
2,3,102,3,1
3,4,103,4,1
4,5,104,5,4
5,6,105,1,2


### QUIZ

1. List all customers and countries
2. List all products in the 'Furniture' category
3. Find all orders made by 'Alice'
4. Show each order with the customer name and order date
5. List all items in order '101' with product name and quantity
6. Count how many products there are in each category
7. Find the total quantity of items ordered by each customer
8. Find the total spending of each customer
9. Show customers who bought products from more than one category
10. List the top 2 customers who spent the most

In [77]:
# 1) List all customers and countries

query = \
"""
SELECT name, country 
FROM customers;
"""

run_query(query)

,name,country
0,Alice,USA
1,Bob,UK
2,Charlie,USA
3,Diana,Germany
4,Eve,Spain


In [78]:
# 2)List all products in the 'Furniture' category

query = \
"""
SELECT *
FROM products
WHERE category = 'Furniture';
"""

run_query(query)

,prod_id,name,category,price
0,4,Desk,Furniture,200.0
1,5,Chair,Furniture,100.0


In [79]:
# 3) Find all orders made by 'Alice'

query = \
"""
SELECT c.name, o.* 
FROM customers c JOIN orders o 
ON c.cust_id = o.cust_id
WHERE c.name = 'Alice'
"""

run_query(query)

,name,order_id,cust_id,order_date,status
0,Alice,101,1,2023-01-10,Completed
1,Alice,103,1,2023-02-01,Pending


In [80]:
# 4) Show each order with the customer name and order date

query = \
"""
SELECT c.name, o.order_date
FROM customers c JOIN orders o 
ON c.cust_id = o.cust_id
"""

run_query(query)

,name,order_date
0,Alice,2023-01-10
1,Bob,2023-01-12
2,Alice,2023-02-01
3,Charlie,2023-02-05
4,Diana,2023-02-20


In [81]:
# 5) List all items in order 101 with product name and quantity

query = \
"""
SELECT p.name, o.quantity
FROM order_items o JOIN products p
ON o.prod_id = p.prod_id
WHERE o.order_id = 101;
"""

run_query(query)

,name,quantity
0,Laptop,1
1,Headphones,2


In [82]:
# 6) Count how many products there are in each category.

query = \
"""
SELECT category, COUNT(*) as count
FROM products
GROUP BY category;
"""

run_query(query)

,category,count
0,Electronics,2
1,Furniture,2
2,Home,1


In [83]:
# 7) Find the total quantity of items ordered by each customer

query = \
"""
SELECT c.name, SUM(oi.quantity)
FROM customers c 
JOIN orders o ON c.cust_id = o.cust_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.name;
"""

run_query(query)

,name,SUM(oi.quantity)
0,Alice,4
1,Bob,1
2,Charlie,4
3,Diana,2


In [84]:
# 8) Find the total spending of each customer

query = \
"""
SELECT c.name, SUM(oi.quantity * p.price) as spending
FROM customers c 
JOIN orders o ON c.cust_id = o.cust_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.prod_id = p.prod_id
GROUP BY c.name;
"""

run_query(query)

,name,spending
0,Alice,1700.0
1,Bob,300.0
2,Charlie,400.0
3,Diana,2400.0


In [85]:
# 9) Show customers who bought products from more than one category

query = \
"""
SELECT c.name
FROM customers c
JOIN orders o on c.cust_id = o.cust_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON p.prod_id = oi.prod_id
GROUP BY c.name
HAVING COUNT(DISTINCT p.category) > 1; 
"""

run_query(query)

,name
0,Alice


In [86]:
# 10) List the top 2 customers who spent the most

query = \
"""
SELECT c.name, SUM(oi.quantity * p.price) as spending
FROM customers c 
JOIN orders o ON c.cust_id = o.cust_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.prod_id = p.prod_id
GROUP BY c.name
ORDER BY spending DESC
LIMIT 2;
"""

run_query(query)

,name,spending
0,Diana,2400.0
1,Alice,1700.0
